In [ ]:
import text2emotion as te
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
import string
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
class MoodAnalysis:
    
     def __init__(self, journal_entry):
        self.journal_entry = journal_entry
        self.t2e_entry_mood = {}
        self.t2e_sent_mood_list = []
        self.nltk_entry_mood = {}
        self.nltk_sent_mood_list = []
        
    def t2e_entry_analysis:
        self.t2e_entry_mood = te.get_emotion(journal_entry.entry)
    
    def t2e_sent_analysis:
        sent_tokenized = {}
        sent_tokenized = nltk.tokenize.sent_tokenize(journal_entry.entry)
        for sentence in sent_tokenized:
            mood = te.get_emotion(sentence)
            self.t2e_sent_mood_list.append(mood)
    
    def nltk_entry_analysis:
        sia = SentimentIntensityAnalyzer()
        nltk_entry_mood = sia.polarity_scores(journal_entry.entry)

    def nltk_sent_analysis:
        sia = SentimentIntensityAnalyzer()
        sent_tokenized = {}
        sent_tokenized = nltk.tokenize.sent_tokenize(journal_entry.entry)
        for sentence in sent_tokenized:
            mood = sia.polarity_scores(sentence)
            self.nltk_sent_mood_list.append(mood)
    